In [3]:
#加载包
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,1.4158,2.9711,10.7935,7.5279,2.3352,8.1042,2.3096,3.3367,11.8639,12.7142,...,171.764,1434.24,0.331511,-0.932553,0.285048,-0.1435,-0.833982,0.767568,0.463969,1.904800
1,0.6280,1.8616,10.1770,7.4684,2.1915,8.5945,0.1379,2.9661,11.5816,12.2487,...,185.824,1469.19,0.894066,-0.446796,0.058519,-0.4624,0.715252,0.999105,0.988844,0.689742
2,0.9648,1.8103,10.1682,5.9705,2.0629,6.5349,2.8694,3.1185,11.7464,12.2074,...,187.576,1540.76,0.999982,0.460716,0.997809,-0.4624,0.723031,0.992935,0.682903,0.749580
3,0.7119,1.6221,10.1487,6.8678,2.0694,6.8806,1.5791,2.3003,11.5545,12.0659,...,189.938,1498.29,0.998794,-0.862448,0.329694,-0.4624,0.891745,0.015078,0.997127,0.984439
4,0.3797,1.6852,10.9601,5.0035,3.1659,5.9471,0.0858,2.6402,11.7458,12.7041,...,181.275,1465.11,0.442893,-0.990703,-0.151400,0.2245,0.617880,-0.506397,-0.997502,0.376126


In [12]:
# 读取CSV文件
data = pd.read_csv('data.csv')

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,特征10,特征11,特征12,特征13,特征14,特征15,特征16,特征17
0,1.4158,2.9711,10.7935,7.5279,2.3352,8.1042,2.3096,3.3367,11.8639,12.7142,1.8581,0.3898,19.8309,19.7710,0.0001,1.7768,171.764,1434.24
1,0.6280,1.8616,10.1770,7.4684,2.1915,8.5945,0.1379,2.9661,11.5816,12.2487,1.9536,1.3220,19.2644,19.4882,3.0496,1.8356,185.824,1469.19
2,0.9648,1.8103,10.1682,5.9705,2.0629,6.5349,2.8694,3.1185,11.7464,12.2074,1.8294,1.2046,18.4731,19.4309,1.8249,28.7651,187.576,1540.76
3,0.7119,1.6221,10.1487,6.8678,2.0694,6.8806,1.5791,2.3003,11.5545,12.0659,1.9988,1.5685,18.5662,19.0838,6.6821,9.3792,189.938,1498.29
4,0.3797,1.6852,10.9601,5.0035,3.1659,5.9471,0.0858,2.6402,11.7458,12.7041,1.9722,1.3629,19.0996,18.8443,8.1784,4.7450,181.275,1465.11


In [17]:
# 获取特征和标签列
features = data.iloc[:, :18].values
labels = data.iloc[:, 18].values#更换18、19、20、21、22、23、24、25分别对应补偿0-7

# 对特征和标签进行归一化处理
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)
labels_scaled = labels.reshape(-1,1)

C:\Users\80521\AppData\Local\Temp\ipykernel_14296\2197065348.py:20: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


[[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.39570027e-109
  0.00000000e+000 1.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.39570027e-109
  0.00000000e+000 1.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.39570027e-109
  0.00000000e+000 1.00000000e+000]
 ...
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.39570027e-109
  0.00000000e+000 1.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.39570027e-109
  0.00000000e+000 1.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.39570027e-109
  0.00000000e+000 1.00000000e+000]]


In [18]:
# 构建训练集、验证集和测试集
train_size = int(len(features_scaled) * 0.8)#总行数的70%为训练集
val_size = int(len(features_scaled) * 0.15)#10%为验证集
test_size = len(features_scaled) - train_size - val_size#剩下20%为测试集
train_features = features_scaled[:train_size, :]
train_labels = labels_scaled[:train_size]
val_features = features_scaled[train_size:train_size+val_size, :]
val_labels = labels_scaled[train_size:train_size+val_size]
test_features = features_scaled[train_size+val_size:, :]
test_labels = labels_scaled[train_size+val_size:]

ERROR =  0.5535468972766894


In [ ]:
# 将数据转换为LSTM输入格式
def create_lstm_dataset(features, labels, time_steps=1):
    dataX, dataY = [], []
    for i in range(len(features) - time_steps):
        a = features[i:(i + time_steps), :]
        dataX.append(a)
        dataY.append(labels[i + time_steps])
    return np.array(dataX), np.array(dataY)

time_steps = 2#可调
trainX, trainY = create_lstm_dataset(train_features, train_labels, time_steps)
valX, valY = create_lstm_dataset(val_features, val_labels, time_steps)
testX, testY = create_lstm_dataset(test_features, test_labels, time_steps)


In [ ]:
# 构建LSTM模型
model = Sequential()
model.add(LSTM(50,activation='sigmoid', input_shape=(time_steps, 18)))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# 拟合模型
history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=100, batch_size=64, verbose=1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
# 绘制损失曲线
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# 预测测试集
test_predict = model.predict(testX)
# 将预测值和真实值绘图展示
plt.plot(test_predict, label='Predicted')
plt.plot(testY, label='True')
plt.legend()
plt.show()

In [ ]:
#精度评价
MSE = mean_squared_error(test_predict,testY)
print ("均方误差为：", MSE)